In [215]:
#Load the necessary Libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',100)

loan = pd.read_csv('loan.csv')
loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,...,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,NaN,NaN,3,0,13648,83.70%,9,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,NaN,May-16,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0,Apr-99,5,NaN,NaN,3,0,1687,9.40%,4,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,NaN,NaN,2,0,2956,98.50%,10,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,NaN,May-16,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0,Feb-96,1,35.0,NaN,10,0,5598,21%,37,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,NaN,Apr-16,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,

# 1. Data Understanding

In [216]:
# CHecking total no of columns and rows
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


In [217]:
# Checking all the columns with null values
loan.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64

In [218]:
# CHecking if any amount related columns have null values. It is needed because thse columns will be used in decision taking
loan[['loan_amnt','funded_amnt','funded_amnt_inv','term','int_rate','installment']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   loan_amnt        39717 non-null  int64  
 1   funded_amnt      39717 non-null  int64  
 2   funded_amnt_inv  39717 non-null  float64
 3   term             39717 non-null  object 
 4   int_rate         39717 non-null  object 
 5   installment      39717 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 1.8+ MB


In [219]:
loan.shape

(39717, 111)

### Checking columns having more than 60% values null

In [220]:

missing_values = round((loan.isnull().sum()/len(loan.id))*100, 2)
#Columns where all the values are missing/NaN. Need to check if this has any importance
#missing_values = missing_values[(missing_values==100)]
#missing_values = missing_values[(missing_values>0) & (missing_values<100)]
missing_values.sort_values(ascending=False)

missing_values_60_percent = missing_values[missing_values>60]
missing_values_60_percent

mths_since_last_delinq             64.66
mths_since_last_record             92.99
next_pymnt_d                       97.13
mths_since_last_major_derog       100.00
annual_inc_joint                  100.00
dti_joint                         100.00
verification_status_joint         100.00
tot_coll_amt                      100.00
tot_cur_bal                       100.00
open_acc_6m                       100.00
open_il_6m                        100.00
open_il_12m                       100.00
open_il_24m                       100.00
mths_since_rcnt_il                100.00
total_bal_il                      100.00
il_util                           100.00
open_rv_12m                       100.00
open_rv_24m                       100.00
max_bal_bc                        100.00
all_util                          100.00
total_rev_hi_lim                  100.00
inq_fi                            100.00
total_cu_tl                       100.00
inq_last_12m                      100.00
acc_open_past_24

In [221]:
# CHecking if any row is empty
loan2.isnull().all(axis=1).sum()

0

# 2. Data Cleaning and Manipulation

In [222]:
# We see several values where more than 60% values are missing. Lets keep a threshold for 60% missing values
# and remove all the columns where more than 60% values are missing
loan = loan.drop(missing_values_60_percent.index, axis=1)
loan.shape

(39717, 54)

In [205]:
# Analyse missing values again after removing the threshold columns
missing_values = round((loan.isnull().sum()/len(loan.id))*100, 2)
missing_values = pd.DataFrame(missing_values[missing_values>0], columns=["missing_value_percentage"])
missing_values.sort_values(by=["missing_value_percentage"], ascending=False)

,missing_value_percentage
desc,32.58
emp_title,6.19
emp_length,2.71
pub_rec_bankruptcies,1.75
last_pymnt_d,0.18
collections_12_mths_ex_med,0.14
chargeoff_within_12_mths,0.14
revol_util,0.13
tax_liens,0.10
title,0.03


In [206]:
# Also, desc column has no role on decision so lets remove that as well.
# Reason behind the removal: 1. 33% columns have no desc, also after going through some of the desc values it seems it just 
#a basic summary and borrower added date that can be find from other columns as well eg. purpose
loan = loan.drop('desc', axis=1)
loan.shape

(39717, 53)

### Filtering the columns which have same values for all the rows/records. Dropping the columns

In [223]:
# Check for same values
loan.nunique().sort_values(ascending=True)

tax_liens                         1
delinq_amnt                       1
chargeoff_within_12_mths          1
acc_now_delinq                    1
application_type                  1
policy_code                       1
initial_list_status               1
collections_12_mths_ex_med        1
pymnt_plan                        1
term                              2
pub_rec_bankruptcies              3
verification_status               3
loan_status                       3
pub_rec                           5
home_ownership                    5
grade                             7
inq_last_6mths                    9
emp_length                       11
delinq_2yrs                      11
purpose                          14
sub_grade                        35
open_acc                         40
addr_state                       50
issue_d                          55
total_acc                        82
last_pymnt_d                    101
last_credit_pull_d              106
int_rate                    

In [210]:
# Columns having only single values
columns_with_single_value = ['tax_liens', 'delinq_amnt', 'chargeoff_within_12_mths', 'acc_now_delinq', 'application_type', 
                             'policy_code', 'collections_12_mths_ex_med', 'initial_list_status', 'pymnt_plan' ]
loan[columns_with_single_value]
# Meaning of the values and decide if the columns can be removed
#tax_liens : a legal claim against the assets of a person or business who fails to pay taxes owed. 
            #here all the values are 0 or NaN, so no role in the model. Needs to be removed
# delinq_amnt: Since the is no any past due amount for any account id, so we can remove this column
#chargeoff_within_12_mths : cahrge off means, a lender or creditor has written the account off as a loss, 
                           #and the account is closed to future charges. There is no any, so we can remove the column
#acc_now_delinq : It means the account on which the lender is delinquent. Since no any, sow e can remove column
#application_type : Here all the application_type is INDIVIDUAL only so we can remove it from model making.
#policy_code : Here each row is for publicly available policy_code, so we can remove it from model making.
#collections_12_mths_ex_med : No of collection in last 12 months including medical collection are either 0 or NaN, we can remove this column
#initial_list_status : The initial loan status can be W: Whole loan, F: Fractional Purchage. Here every valiue is f. We can remove column
#pymnt_plan : It indicates if a payment plan has been put in place for the loan. here every value is n i.e No

# Summary: If needed We can take the common value for each records, application_type= INDIVIDUAL, initial_list_status= f, pymnt_plan=n if needed further

,tax_liens,delinq_amnt,chargeoff_within_12_mths,acc_now_delinq,application_type,policy_code,collections_12_mths_ex_med,initial_list_status,pymnt_plan
0,0.0,0,0.0,0,INDIVIDUAL,1,0.0,f,n
1,0.0,0,0.0,0,INDIVIDUAL,1,0.0,f,n
2,0.0,0,0.0,0,INDIVIDUAL,1,0.0,f,n
3,0.0,0,0.0,0,INDIVIDUAL,1,0.0,f,n
4,0.0,0,0.0,0,INDIVIDUAL,1,0.0,f,n
...,...,...,...,...,...,...,...,...,...
39712,NaN,0,NaN,0,INDIVIDUAL,1,NaN,f,n
39713,NaN,0,NaN,0,INDIVIDUAL,1,NaN,f,n
39714,NaN,0,NaN,0,INDIVIDUAL,1,NaN,f,n
39715,NaN,0,NaN,0,INDIVIDUAL,1,NaN,f,n


In [224]:
# Remove above columns
loan = loan.drop(columns_with_single_value, axis=1)
loan.shape

(39717, 45)

### Correcting the data by ommiting unnecessary prefixes and suffixes


In [212]:
loan.dtypes

id                           int64
member_id                    int64
loan_amnt                    int64
funded_amnt                  int64
funded_amnt_inv            float64
term                        object
int_rate                    object
installment                float64
grade                       object
sub_grade                   object
emp_title                   object
emp_length                  object
home_ownership              object
annual_inc                 float64
verification_status         object
issue_d                     object
loan_status                 object
url                         object
purpose                     object
title                       object
zip_code                    object
addr_state                  object
dti                        float64
delinq_2yrs                  int64
earliest_cr_line            object
inq_last_6mths               int64
open_acc                     int64
pub_rec                      int64
revol_bal           

In [228]:
loan['int_rate'] = loan['int_rate'].apply(lambda x: x[:-1])
loan['term'] = loan['term'].apply(lambda x: x[:-7])
loan[['int_rate','term']].head()

,int_rate,term
0,10.65,36
1,15.27,60
2,15.96,36
3,13.49,36
4,12.69,60


In [229]:
# emp_length is for the duration a lender is employed. It can play a good role in deciding the loan approve/rejection
len(loan['emp_length'][loan['emp_length'].isnull()]) # 1075

# FIll with zero if there is any NaN value
loan['emp_length'].fillna('0',inplace=True)
len(loan['emp_length'][loan['emp_length'].isnull()])   # Now 0
# Remove all the extra valueas like years, < , > or + signs and keep only numeric value
loan['emp_length'] = loan['emp_length'].str.extract('(\d+)')
loan['emp_length']

0        10
1         1
2        10
3        10
4         1
         ..
39712     4
39713     3
39714     1
39715     1
39716     1
Name: emp_length, Length: 39717, dtype: object

In [230]:
loan.dtypes

id                           int64
member_id                    int64
loan_amnt                    int64
funded_amnt                  int64
funded_amnt_inv            float64
term                        object
int_rate                    object
installment                float64
grade                       object
sub_grade                   object
emp_title                   object
emp_length                  object
home_ownership              object
annual_inc                 float64
verification_status         object
issue_d                     object
loan_status                 object
url                         object
desc                        object
purpose                     object
title                       object
zip_code                    object
addr_state                  object
dti                        float64
delinq_2yrs                  int64
earliest_cr_line            object
inq_last_6mths               int64
open_acc                     int64
pub_rec             

In [231]:
#Changing Data types for term, int_rate and emp_length to float
loan['term'] = loan['term'].astype('float64')
loan['int_rate'] = loan['int_rate'].astype('float64')
loan['emp_length'] = loan['emp_length'].astype('float64')
loan.dtypes

id                           int64
member_id                    int64
loan_amnt                    int64
funded_amnt                  int64
funded_amnt_inv            float64
term                       float64
int_rate                   float64
installment                float64
grade                       object
sub_grade                   object
emp_title                   object
emp_length                 float64
home_ownership              object
annual_inc                 float64
verification_status         object
issue_d                     object
loan_status                 object
url                         object
desc                        object
purpose                     object
title                       object
zip_code                    object
addr_state                  object
dti                        float64
delinq_2yrs                  int64
earliest_cr_line            object
inq_last_6mths               int64
open_acc                     int64
pub_rec             

### Date columns cleaning and adding derived columns related to date


In [233]:
loan[['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']]

,issue_d,earliest_cr_line,last_pymnt_d,last_credit_pull_d
0,Dec-11,Jan-85,Jan-15,May-16
1,Dec-11,Apr-99,Apr-13,Sep-13
2,Dec-11,Nov-01,Jun-14,May-16
3,Dec-11,Feb-96,Jan-15,Apr-16
4,Dec-11,Jan-96,May-16,May-16
...,...,...,...,...
39712,Jul-07,Nov-90,Jul-10,Jun-10
39713,Jul-07,Dec-86,Jul-10,Jul-10
39714,Jul-07,Oct-98,Apr-08,Jun-07
39715,Jul-07,Nov-88,Jan-08,Jun-07


In [234]:
loan['issue_d'] = loan['issue_d'].apply(pd.to_datetime, format='%b-%y')
loan['earliest_cr_line'] = loan['earliest_cr_line'].apply(pd.to_datetime, format='%b-%y')
loan['last_pymnt_d'] = loan['last_pymnt_d'].apply(pd.to_datetime, format='%b-%y')
loan['last_credit_pull_d'] = loan['last_credit_pull_d'].apply(pd.to_datetime, format='%b-%y')
loan[['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']].dtypes

issue_d               datetime64[ns]
earliest_cr_line      datetime64[ns]
last_pymnt_d          datetime64[ns]
last_credit_pull_d    datetime64[ns]
dtype: object

In [235]:
# Derived columns for dates
loan['issue_year']=loan['issue_d'].dt.year
loan['issue_month']=loan['issue_d'].dt.month
loan

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies,issue_year,issue_month
0,1077501,1296599,5000,5000,4975.0,36.0,10.65,162.87,B,B2,NaN,10.0,RENT,24000.0,Verified,2011-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0,1985-01-01,1,3,0,13648,83.70%,9,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,2015-01-01,171.62,2016-05-01,0.0,2011,12
1,1077430,1314167,2500,2500,2500.0,60.0,15.27,59.83,C,C4,Ryder,1.0,RENT,30000.0,Source Verified,2011-12-01,Charged Off,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0,1999-04-01,5,3,0,1687,9.40%,4,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,2013-04-01,119.66,2013-09-01,0.0,2011,12
2,1077175,1313524,2400,2400,2400.0,36.0,15.96,84.33,C,C5,NaN,10.0,RENT,12252.0,Not Verified,2011-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0,2001-11-01,2,2,0,2956,98.50%,10,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,2014-06-01,649.91,2016-05-01,0.0,2011,12
3,1076863,1277178,10000,10000,10000.0,36.0,13.49,339.31,C,C1,AIR RESOURCES BOARD,10.0,RENT,49200.0,Source Verified,2011-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0,1996-02-01,1,10,0,5598,21%,37,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,2015-01-01,357.48,2016-04-01,0.0,2011,12
4,1075358,1311748,3000,3000,3000.0,60.0,12.69,67.79,B,B5,University Medical Group,1.0,RENT,80000.0,Source Verified,2011-12-01,Current,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0,1996-01-01,0,15,0,27783,53.90%,38,524.06,524.06,3513.330000,3513.33,2475.94,1037.39,0.00,0.00,0.00,2016-05-01,67.79,2016-05-01,0.0,2011,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,92187,92174,2500,2500,1075.0,36.0,8.07,78.42,A,A4,FiSite Research,4.0,MORTGAGE,110000.0,Not Verified,2007-07-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,Our current gutter system on our home is old a...,home_improvement,Home Improvement,802xx,CO,11.33,0,1990-11-01,0,13,0,7274,13.10%,40,0.00,0.00,2822.969293,1213.88,2500.00,322.97,0.00,0.00,0.00,2010-07-01,80.90,2010-06-01,NaN,2007,7
39713,90665,90607,8500,8500,875.0,36.0,10.28,275.38,C,C1,"Squarewave Solutions, Ltd.",3.0,RENT,18000.0,Not Verified,2007-07-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,The rate of interest and fees incurred by carr...,credit_card,Retiring credit card debt,274xx,NC,6.40,1,1986-12-01,1,6,0,8847,26.90%,9,0.00,0.00,9913.491822,1020.51,8500.00,1413.49,0.00,0.00,0.00,2010-07-01,281.94,2010-07-01,NaN,2007,7
39714,90395,90390,5000,5000,1325.0,36.0,8.07,156.84,A,A4,NaN,1.0,MORTGAGE,100000.0,Not Verified,2007-07-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,MBA Loan Consolidation,017xx,MA,2.30,0,1998-10-01,0,11,0,9698,19.40%,20,0.00,0.00,5272.161128,1397.12,5000.00,272.16,0.00,0.00,0.00,2008-04-01,0.00,2007-06-01,NaN,2007,7
39715,90376,89243,5000,5000,650.0,36.0,7.43,155.38,A,A2,NaN,1.0,MORTGAGE,200000.0,Not Verified,2007-07-01,Fully Paid,https://lendingclub.com/browse/lo

# 3. Data analysis